In [1074]:
!git clone https://github.com/victorgfb/brazilian-coin-detector.git

fatal: destination path 'brazilian-coin-detector' already exists and is not an empty directory.


In [1075]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from skimage import data, exposure, img_as_float

In [1076]:
data_dir = "brazilian-coin-detector/newDataset/coins"

In [1077]:

data_gen = ImageDataGenerator(rescale=1/255. , horizontal_flip=True, vertical_flip = True, validation_split=0.2)

In [1078]:
train_generator = data_gen.flow_from_directory(data_dir, target_size=(96,96), class_mode="categorical", shuffle=True, subset = 'training')
test_generator = data_gen.flow_from_directory(data_dir,target_size=(96,96), class_mode="categorical", shuffle=True, subset = 'validation')

Found 1969 images belonging to 5 classes.
Found 489 images belonging to 5 classes.


In [1079]:
img_shape = (96, 96, 3)

In [1080]:
# base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
#                                                include_top = False,
#                                                weights = "imagenet")

In [1081]:
base_model = tf.keras.applications.ResNet50V2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

In [1082]:
# base_model = tf.keras.applications.VGG16(input_shape = img_shape, 
#                                                include_top = False,
#                                                weights = "imagenet")

In [1083]:
# base_model.summary()

In [1084]:
model = tf.keras.Sequential()

In [1085]:
model.add(base_model)
# model.add(tf.keras.layers.Flatten(name="flatten"))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
# model.add(tf.keras.layers.Dense(60, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [1086]:
len(model.predict(test_generator))

489

In [1087]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 3, 3, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_28  (None, 2048)              0         
_________________________________________________________________
dense_93 (Dense)             (None, 120)               245880    
_________________________________________________________________
dropout_65 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_94 (Dense)             (None, 120)               14520     
_________________________________________________________________
dropout_66 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_95 (Dense)             (None, 5)               

In [1088]:
# model.add(base_model)
# model.add(tf.keras.layers.Flatten(name="flatten"))
# model.add(tf.keras.layers.Dense(256, activation="relu"))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(256, activation="relu"))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(5, activation="softmax"))

In [1089]:
len(base_model.layers)

190

In [1090]:
base_model.trainable = True

In [1091]:
fine_tuning_at = 25

In [1092]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

In [1093]:
callback = tf.keras.callbacks.EarlyStopping(patience = 3)

In [1094]:
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [1095]:
# lr_schedule = 0.001

In [1096]:
loss = tf.keras.losses.CategoricalCrossentropy()
# loss = softmax_cross_entropy_with_logits
# loss=tf.keras.losses.SparseCategoricalCrossentropy()

In [1097]:
model.compile(optimizer=optimizer, loss = loss, metrics=['categorical_accuracy'])

In [1098]:
history = model.fit(train_generator, epochs=5, validation_data=test_generator, callbacks=[callback])

Epoch 1/5
62/62 [==============================] - 12s 194ms/step - loss: 1.1863 - categorical_accuracy: 0.5175 - val_loss: 0.8305 - val_categorical_accuracy: 0.6462
Epoch 2/5
62/62 [==============================] - 11s 178ms/step - loss: 0.3627 - categorical_accuracy: 0.8949 - val_loss: 0.7354 - val_categorical_accuracy: 0.7526
Epoch 3/5
62/62 [==============================] - 11s 178ms/step - loss: 0.1791 - categorical_accuracy: 0.9492 - val_loss: 0.3903 - val_categorical_accuracy: 0.8875
Epoch 4/5
62/62 [==============================] - 11s 177ms/step - loss: 0.1084 - categorical_accuracy: 0.9746 - val_loss: 0.3996 - val_categorical_accuracy: 0.8630
Epoch 5/5
62/62 [==============================] - 11s 178ms/step - loss: 0.0783 - categorical_accuracy: 0.9761 - val_loss: 0.2995 - val_categorical_accuracy: 0.9182


In [1099]:
model.save('my_model.h5')

In [1118]:
import cv2
import numpy as np
img = cv2.imread("2.jpg")
im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [1119]:
img_res = cv2.resize(im_rgb, (96,96))

In [1120]:
norm_img = img_res/255

In [1121]:
np_img = np.expand_dims(norm_img, axis=0)

In [1122]:
test = model.predict(np_img)

In [1123]:
test

array([[1.4039830e-06, 4.6254995e-06, 9.9999130e-01, 2.0816196e-06,
        5.5493689e-07]], dtype=float32)

In [1124]:
coinsType = list(test_generator.class_indices.keys())

In [1125]:
coinsType[int(np.argmax(test, axis=1))]

'25'

In [1126]:
test_generator.class_indices.keys()

dict_keys(['10', '100', '25', '5', '50'])

TODO

colocar para plotar grafico de aprendizado
